<a href="https://colab.research.google.com/github/kazeidk/GDrive_Turbo_Copy/blob/main/GDrive_Turbo_Copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 GDrive Turbo Copy - Stable Edition

## ✨ Tính năng:
- 🎯 **Exact match** - Kiểm tra file chính xác
- 🔄 **Auto-resume** - Tự tiếp tục khi timeout
- 🔁 **Auto-retry** - Tự retry khi rate limit
- 📊 **Progress bar** - Hiển thị tiến độ
- 🔍 **Filter** - Lọc theo đuôi file
- 💾 **Checkpoint** - Lưu tiến độ mỗi 10 files

---

## 📖 Hướng dẫn:
1. Chạy cell **Cài đặt**
2. Chạy cell **Nhập thông tin**
3. Chạy cell **Run**

## 🔥 Recommend setting Colab Pro+:
- GPU: **A100** | RAM cao: **BẬT**

In [ ]:
#@title 1️⃣ Cài đặt thư viện
!pip install tqdm -q
print("✅ Đã cài đặt xong!")

In [ ]:
#@title 2️⃣ Nhập thông tin
from ipywidgets import widgets, Layout, HBox
from IPython.display import display, HTML
import psutil

# Detect system info
try:
    ram_gb = psutil.virtual_memory().total / (1024**3)
    tier = 'Pro+' if ram_gb > 50 else ('Pro' if ram_gb > 12 else 'Free')
except:
    ram_gb = 0
    tier = 'Unknown'

try:
    import subprocess
    gpu_info = subprocess.check_output(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader']).decode().strip()
except:
    gpu_info = 'No GPU'

print('🔥' + '='*48 + '🔥')
print(f'🚀 DETECTED: {tier.upper()}')
print(f'💾 System RAM: {ram_gb:.1f} GB')
print(f'🎮 GPU: {gpu_info}')
print('🔥' + '='*48 + '🔥')

style = {'description_width': '180px'}
layout = Layout(width='600px')

dest_text = widgets.Text(description='📁 Folder đích:', placeholder='Link folder Google Drive của bạn', style=style, layout=layout)
source_text = widgets.Text(description='📂 Folder nguồn:', placeholder='Link folder cần copy', style=style, layout=layout)
max_download_size_text = widgets.Text(description='💾 Giới hạn (GB):', value='0', placeholder='0 = không giới hạn', style=style, layout=layout)
exclude_str_text = widgets.Text(description='🚫 Bỏ qua chứa:', value='', placeholder='.tmp, backup', style=style, layout=layout)
include_ext_text = widgets.Text(description='✅ Chỉ copy đuôi:', value='', placeholder='.pdf, .mp4', style=style, layout=layout)
exclude_ext_text = widgets.Text(description='🚫 Bỏ qua đuôi:', value='', placeholder='.tmp, .log', style=style, layout=layout)
dry_run_checkbox = widgets.Checkbox(description='🔍 Dry-run (chỉ xem)', value=False, layout=Layout(width='300px'))
overwrite_checkbox = widgets.Checkbox(description='📝 Ghi đè nếu size khác', value=False, layout=Layout(width='300px'))

display(HTML('<b>📌 Thông tin chính:</b>'))
display(dest_text, source_text)
display(HTML('<b>⚙️ Cấu hình:</b>'))
display(max_download_size_text)
display(HTML('<b>🔍 Bộ lọc:</b>'))
display(exclude_str_text, include_ext_text, exclude_ext_text)
display(HTML('<b>🎛️ Tùy chọn:</b>'))
display(dry_run_checkbox, overwrite_checkbox)
print('-' * 50)
print('✅ Nhập xong thì chạy cell tiếp theo!')

In [ ]:
#@title 3️⃣ Run - GDrive Turbo Copy
import os
import time
import re
import json
import gc
from datetime import datetime
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.colab import auth
from tqdm.notebook import tqdm

class GDriveTurboCopy:
    def __init__(self):
        self._total_size = 0
        self._limit_size = 0
        self._copied_count = 0
        self._skipped_count = 0
        self._error_count = 0
        self.excluded_strings = []
        self.include_ext = []
        self.exclude_ext = []
        self.dry_run = False
        self.overwrite = False
        self.checkpoint = '/content/copy_checkpoint.json'
        self.logfile = '/content/copy_log.txt'
        self.copied_files = set()
        self.service = None

    def log(self, msg):
        ts = datetime.now().strftime('%H:%M:%S')
        print(f'[{ts}] {msg}')
        try:
            with open(self.logfile, 'a', encoding='utf-8') as f:
                f.write(f'[{datetime.now()}] {msg}\n')
        except: pass

    def load_checkpoint(self):
        try:
            if os.path.exists(self.checkpoint):
                with open(self.checkpoint, 'r') as f:
                    d = json.load(f)
                    self.copied_files = set(d.get('files', []))
                    self._total_size = d.get('size', 0)
                    self._copied_count = d.get('copied', 0)
                    self._skipped_count = d.get('skipped', 0)
                    self.log(f'✅ Resume: {len(self.copied_files)} files, {self._total_size/1024:.2f}GB')
        except: pass

    def save_checkpoint(self):
        try:
            with open(self.checkpoint, 'w') as f:
                json.dump({'files': list(self.copied_files), 'size': self._total_size, 'copied': self._copied_count, 'skipped': self._skipped_count}, f)
        except: pass

    def auth(self):
        self.log('🔐 Đang xác thực...')
        auth.authenticate_user()
        self.service = build('drive', 'v3')
        self.log('✅ Xác thực thành công')

    def sanitize(self, name):
        if not name: return 'unnamed'
        for c in ['/', '\\', ':', '*', '?', '"', '<', '>', '|']:
            name = name.replace(c, '_')
        return name[:200].strip() or 'unnamed'

    def should_include(self, name):
        if not name: return True
        ext = os.path.splitext(name)[1].lower()
        if self.include_ext: return ext in self.include_ext
        if self.exclude_ext: return ext not in self.exclude_ext
        return True

    def api_call(self, func, retries=3):
        for i in range(retries):
            try:
                return func()
            except HttpError as e:
                if e.resp.status in [403, 429, 500, 502, 503]:
                    wait = min(5 * (2 ** i), 60)
                    self.log(f'⚠️ API error {e.resp.status}, retry in {wait}s...')
                    time.sleep(wait)
                elif e.resp.status == 404:
                    return None
                else:
                    raise
            except Exception as e:
                if i < retries - 1:
                    time.sleep(5)
                else:
                    raise
        return None

    def list_folder(self, folder_id):
        files = []
        token = None
        q = f"'{folder_id}' in parents and trashed=false"
        for s in self.excluded_strings:
            q += f" and not name contains '{s}'"

        while True:
            try:
                r = self.api_call(lambda: self.service.files().list(
                    q=q, pageToken=token, pageSize=1000,
                    fields='files(id,name,mimeType,size),nextPageToken',
                    supportsAllDrives=True, includeItemsFromAllDrives=True
                ).execute())
                if not r: break

                for f in r.get('files', []):
                    mime = f.get('mimeType', '')
                    if mime == 'application/vnd.google-apps.folder':
                        files.append(f)
                    elif self.should_include(f.get('name', '')):
                        if not mime.startswith('application/vnd.google-apps.'):
                            files.append(f)

                token = r.get('nextPageToken')
                if not token: break
            except Exception as e:
                self.log(f'❌ List error: {e}')
                break
        return files

    def file_exists(self, dest_id, name, size=None):
        try:
            esc = name.replace("'", "\\'")
            q = f"'{dest_id}' in parents and name='{esc}' and trashed=false"
            r = self.api_call(lambda: self.service.files().list(
                q=q, fields='files(id,size)', supportsAllDrives=True, pageSize=1
            ).execute())
            if r and r.get('files'):
                f = r['files'][0]
                if self.overwrite and size and int(f.get('size', 0)) != size:
                    try: self.service.files().delete(fileId=f['id'], supportsAllDrives=True).execute()
                    except: pass
                    return None
                return f['id']
        except: pass
        return None

    def copy_file(self, dest_id, src):
        name = self.sanitize(src.get('name', 'unnamed'))
        size = int(src.get('size', 0))
        mb = size / (1024*1024)
        key = f"{dest_id}:{src['id']}"

        if key in self.copied_files:
            self._skipped_count += 1
            return 'skip'

        if self.dry_run:
            self.log(f'[DRY] {name} ({mb:.1f}MB)')
            self._total_size += mb
            return 'dry'

        if self.file_exists(dest_id, name, size):
            self._skipped_count += 1
            self.copied_files.add(key)
            return 'exists'

        try:
            t0 = time.time()
            r = self.api_call(lambda: self.service.files().copy(
                body={'parents': [dest_id], 'name': name},
                fileId=src['id'], supportsAllDrives=True
            ).execute())

            if r:
                elapsed = max(time.time() - t0, 0.1)
                speed = mb / elapsed
                self._total_size += mb
                self._copied_count += 1
                self.copied_files.add(key)
                self.log(f'✅ {name} ({mb:.1f}MB, {speed:.1f}MB/s)')

                if self._copied_count % 10 == 0:
                    self.save_checkpoint()
                    gc.collect()

                if self._limit_size > 0 and self._total_size >= self._limit_size * 1024:
                    self.log(f'⚠️ Đạt giới hạn {self._limit_size}GB')
                    return 'limit'
                return 'ok'
        except Exception as e:
            self._error_count += 1
            self.log(f'❌ {name}: {str(e)[:80]}')
        return 'error'

    def create_folder(self, dest_id, name):
        name = self.sanitize(name)
        eid = self.file_exists(dest_id, name)
        if eid: return eid
        if self.dry_run: return f'dry_{name}'

        try:
            r = self.api_call(lambda: self.service.files().create(
                body={'name': name, 'mimeType': 'application/vnd.google-apps.folder', 'parents': [dest_id]},
                fields='id', supportsAllDrives=True
            ).execute())
            if r:
                self.log(f'📁 {name}')
                return r['id']
        except Exception as e:
            self.log(f'❌ Folder {name}: {e}')
        return None

    def copy_folder(self, dest_id, src_id, depth=0):
        items = self.list_folder(src_id)
        if not items: return

        folders = [f for f in items if f.get('mimeType') == 'application/vnd.google-apps.folder']
        files = [f for f in items if f.get('mimeType') != 'application/vnd.google-apps.folder']

        self.log(f'{"  "*depth}📊 {len(files)} files, {len(folders)} folders')

        if files:
            for f in tqdm(files, desc='Copying', leave=False):
                result = self.copy_file(dest_id, f)
                if result == 'limit':
                    return

        for folder in folders:
            fname = folder.get('name', 'unnamed')
            self.log(f'{"  "*depth}📁 {fname}')
            sub_id = self.create_folder(dest_id, fname)
            if sub_id and not sub_id.startswith('dry_'):
                self.copy_folder(sub_id, folder['id'], depth+1)

        gc.collect()

    def extract_id(self, url):
        if not url: return None
        match = re.search(r'[-\w]{25,}', url)
        return match.group(0) if match else None

    def run(self, dest_url, src_url):
        start = time.time()
        self.log('='*50)
        self.log('🚀 GDRIVE TURBO COPY - STABLE')
        self.log('='*50)

        self.auth()
        self.load_checkpoint()

        self.log(f'⚙️ Limit: {self._limit_size}GB | Dry: {self.dry_run}')

        dest_id = self.extract_id(dest_url)
        src_id = self.extract_id(src_url)

        if not dest_id or not src_id:
            self.log('❌ URL không hợp lệ!')
            return

        try:
            src_info = self.service.files().get(fileId=src_id, supportsAllDrives=True).execute()
            dest_info = self.service.files().get(fileId=dest_id, supportsAllDrives=True).execute()
        except Exception as e:
            self.log(f'❌ Không truy cập được folder: {e}')
            return

        self.log(f'📂 Nguồn: {src_info.get("name")}')
        self.log(f'📂 Đích: {dest_info.get("name")}')
        self.log('-'*50)

        new_dest = self.create_folder(dest_id, src_info.get('name', 'Copy'))
        if not new_dest:
            self.log('❌ Không tạo được folder đích!')
            return

        self.copy_folder(new_dest, src_id)
        self.save_checkpoint()

        elapsed = time.time() - start
        gb = self._total_size / 1024
        speed = self._total_size / max(elapsed, 1)

        self.log('='*50)
        self.log('📊 KẾT QUẢ')
        self.log(f'✅ Copied: {self._copied_count} | ⏭️ Skipped: {self._skipped_count} | ❌ Errors: {self._error_count}')
        self.log(f'💾 {gb:.2f}GB | ⏱️ {int(elapsed)}s | 🚀 {speed:.1f}MB/s')
        self.log('='*50)

        if self._error_count == 0:
            self.log('🎉 HOÀN TẤT!')
        else:
            self.log(f'⚠️ Có {self._error_count} lỗi. Chạy lại để retry.')

# === MAIN ===
print("🔧 Khởi tạo...")
dl = GDriveTurboCopy()
dl._limit_size = float(max_download_size_text.value) if max_download_size_text.value else 0
dl.excluded_strings = [s.strip() for s in exclude_str_text.value.split(',') if s.strip()]
dl.include_ext = [s.strip().lower() if s.strip().startswith('.') else f'.{s.strip().lower()}' for s in include_ext_text.value.split(',') if s.strip()]
dl.exclude_ext = [s.strip().lower() if s.strip().startswith('.') else f'.{s.strip().lower()}' for s in exclude_ext_text.value.split(',') if s.strip()]
dl.dry_run = dry_run_checkbox.value
dl.overwrite = overwrite_checkbox.value

if not dest_text.value.strip():
    print("❌ Nhập link folder đích!")
elif not source_text.value.strip():
    print("❌ Nhập link folder nguồn!")
else:
    dl.run(dest_text.value.strip(), source_text.value.strip())

In [ ]:
#@title 4️⃣ Tiện ích - Xóa checkpoint
import os
for f in ['/content/copy_checkpoint.json', '/content/copy_log.txt']:
    if os.path.exists(f):
        os.remove(f)
        print(f'✅ Đã xóa: {f}')
    else:
        print(f'⏭️ Không có: {f}')
print('\n🔄 Có thể chạy lại từ đầu!')